In [6]:
def resample_nifti_to_1mm(input_path, output_path, binary=False):
    """
    Resamplea una imagen NIfTI a una resolución de 1mm x 1mm x 1mm y guarda como un nuevo archivo.
    
    :param input_path: Ruta al archivo NIfTI de entrada.
    :param output_path: Ruta donde se guardará el archivo NIfTI resampleado.
    :param binary: Si es True, convierte el volumen resampleado a binario antes de guardarlo.
    """
    # Carga la imagen
    img_vol = nib.load(input_path)
    img_vol_data = img_vol.get_fdata()
    print("Original data shape:", img_vol_data.shape) # Dimensiones de la imagen
    
    original_affine = img_vol.affine
    
    # Calcula los factores de escala basados en la resolución actual
    original_spacing = np.abs(np.diag(original_affine)[:3])
    
    # Definir el espaciado deseado (1 mm en x, y y 1 mm en z)
    target_spacing = [1.0, 1.0, 1.0]  
    
    # Calcular el factor de escalado en z (y no cambiar en x e y)
    zoom_factors = [1.0,  # x (sin cambios)
                    1.0,  # y (sin cambios)
                    original_spacing[2] / target_spacing[2]]  # z (ajuste)
    
    # Realiza la interpolación para ajustar el tamaño
    resampled_data = zoom(img_vol_data, zoom=zoom_factors, order=1)  # Interpolación lineal
    print("Resampled data shape:", resampled_data.shape)
    
    # Crear el nuevo affine ajustado
    new_affine = np.copy(original_affine)
    new_affine[0, 0] = target_spacing[0] * np.sign(original_affine[0, 0])
    new_affine[1, 1] = target_spacing[1] * np.sign(original_affine[1, 1])
    new_affine[2, 2] = target_spacing[2] * np.sign(original_affine[2, 2])
    
    # Convertir el volumen resampleado a binario si se especifica
    if binary:
        resampled_data = (resampled_data > 0).astype(np.uint8)
        
    # Guarda la imagen con la resolución normalizada
    resampled_img = nib.Nifti1Image(resampled_data, new_affine)
    nib.save(resampled_img, output_path)

In [5]:
def node_coor(node_vol_data, corte):
    """
    Devuelve las coordenadas del nódulo segmentado y la posición media donde se encuentra.
    - Parámetros: 
        node_vol_data (ndarray): Input matricial de la imagen del nodulo segmentado.   
        corte (int): 0 = Corte Sagital, 1 = Corte Coronal, 2 = Corte Axial
        
    - Returns: 
        node_coor_lst (lst): Lista de coordenadas del nodulo en el corte indicado.   
        max_min_list (lst): Lista de dos valores, 1º(número de corte en el que se empieza a observar el nódulo), 2º(número de corte en el que se deja de observar). 
    """
    
    node_coor_lst = []    # Lista coordenadas del nodo
    slice_node = []       # Lista corte
    
    for i in range(node_vol_data.shape[corte]): # Dimensiones 
        # Corte Sagital
        if (corte == 0) and (node_vol_data[i,:,:].max() > 0): # Si hay intensidades positivas, existirá nodo
            # Devolve fila, columna (y,x). 
            node_coor_lst.append(np.argwhere(node_vol_data[i,:,:] > 0)) # Añado coordenadas  
            slice_node.append(i) # Añado slice donde se encuentra el nodo

        # Corte Coronal
        elif (corte == 1) and (node_vol_data[:,i,:].max() > 0):
            node_coor_lst.append(np.argwhere(node_vol_data[:,i,:] > 0))
            slice_node.append(i)  

        # Corte axial
        elif (corte == 2) and (node_vol_data[:,:,i].max() > 0):
            node_coor_lst.append(np.argwhere(node_vol_data[:,:,i] > 0))
            slice_node.append(i)  
            
        else:
            node_coor_lst.append([]) # Añado vacío si no existe nodo
            
    max_min_list = [min(slice_node), max(slice_node)] # Calculamos posición de inicio y de fin del tumor
            
    return node_coor_lst, max_min_list

In [6]:
def boundings_yolo_boxes(node_vol_data, node_coor_lst, corte, img_height, img_width):
    """
    Devuelve las etiquetas normalizadas para la creación de los boundings boxes
    - Parámetros: 
        node_vol_data (ndarray): Input matricial de la imagen del nodulo segmentado.   
        node_coor_lst (lst): Lista de coordenadas del nodulo en el corte indicado.   
        corte (int): 0 = Corte Sagital, 1 = Corte Coronal, 2 = Corte Axial
        img_height (int): Altura de la imagen.
        img_width (in): Anchura de la imagen.

    - Returns:
        labels_norm (list): Lista de las coordenadas del nódulo normalizadas 
    """
    labels_norm = []  # Lista de etiquetas normalizadas
    
    
    for i in range(node_vol_data.shape[corte]): # Dimensiones 
        if len(node_coor_lst[i]) != 0:  # Para cada lista con valor
            
            if corte == 2:  # Corte Axial, 512 x 512. La altura es la misma que la anchura
                width = node_coor_lst[i][:,1].max() - node_coor_lst[i][:,1].min() + 8 # Anchura, xmax - xmin
                width_norm = width / img_height # Normalizo con la altura, 512
    
                x_center = (node_coor_lst[i][:,1].max() + node_coor_lst[i][:,1].min()) / 2 # Centro de x (xmax + xmin) / 2
                x_center_norm = x_center / img_height # Normalizo Normalizo con la altura, 512
                
                height = node_coor_lst[i][:,0].max() - node_coor_lst[i][:,0].min() + 8 # Altura, ymax - ymin
                # Añado 8 px para tener margen de contorno
                height_norm = height / img_height # Normalizo
    
                y_center = (node_coor_lst[i][:,0].max() + node_coor_lst[i][:,0].min()) / 2 # Centro de y (ymax + ymin) / 2
                y_center_norm = y_center / img_height # Normalizo
    
                labels_norm.append([0, x_center_norm, y_center_norm, width_norm, height_norm]) # Guardamos las etiquetas normalizadas
                
            else:  # Corte Coronal y Sagital           
                width = node_coor_lst[i][:,1].max() - node_coor_lst[i][:,1].min() + 8 # Anchura, xmax - xmin
                width_norm = width / img_width # Normalizo
    
                x_center = (node_coor_lst[i][:,1].max() + node_coor_lst[i][:,1].min()) / 2 # Centro de x (xmax + xmin) / 2
                x_center_norm = x_center / img_width # Normalizo
                
                height = node_coor_lst[i][:,0].max() - node_coor_lst[i][:,0].min() + 8 # Altura, ymax - ymin
                # Añado 8 px para tener margen de contorno
                height_norm = height / img_height # Normalizo
    
                y_center = (node_coor_lst[i][:,0].max() + node_coor_lst[i][:,0].min()) / 2 # Centro de y (ymax + ymin) / 2
                y_center_norm = y_center / img_height # Normalizo
    
                labels_norm.append([0, x_center_norm, y_center_norm, width_norm, height_norm]) # Guardamos las etiquetas normalizadas

        else:
            labels_norm.append([]) # Vacío. 
            
    return labels_norm

In [13]:
def save_outputs(img_vol_data, bv_corte, path, corte, max_min_list):
    """
    Guarda tanto las imágenes como las etiquetas correspondientes al corte. 
    - Parámetros:    
        img_vol_data (ndarray): Input matricial de la imagen total del paciente. 
        bv_corte (lst): Lista de las coordenadas del nódulo normalizadas.
        path (str): Directorio de la imagen de input.
        corte (int): 0 = Corte Sagital, 1 = Corte Coronal, 2 = Corte Axial
        max_min_list (lst): Lista de dos valores, 1º(número de corte en el que se empieza a observar el nódulo), 2º(número de corte en el que se deja de observar). 
    """
    
    for i in range(max_min_list[0] - 0, max_min_list[1] + 1): # Cogemos un corte por arriba y por abajo abarcándo el nódulo
        
        if corte == 0:
            # Definimos los paths y los nombres para guardar
            ruta = str(path) + str('/YOLO/Sagital/Images/')
            name = str(path[-3:]) + str('_Sagital_') + str(i) + str('.png')
            ruta_2 = str(path) + str('/YOLO/Sagital/Labels/')
            name_2 = str(path[-3:]) + str('_Sagital_') + str(i) + str('.txt')

            #Enventanamos min= -150, max= 250
            img_w = np.clip(img_vol_data[i,:,:], -150, 250)
            
            #Guardamos normalizada en uint8 en formato .png
            img_norm = (((img_w + 150) / (250 + 150))*255).astype(np.uint8) #Normalizar con escala
            img_n = Image.fromarray(img_norm).convert('L')
            output_path = ruta + name
            img_n.save(output_path)

            txt_clase(ruta_2) # Creamos .txt de clases            
            
            with open(str(ruta_2 + name_2), 'w+') as f_txt: # Creamos el resto de .txt
                for item in (bv_corte[i]): 
                    f_txt.write(str(item) + ' ')
                    
        
        elif corte == 1:
            # Definimos los paths y los nombres para guardar
            ruta = str(path) + str('/YOLO/Coronal/Images/')
            name = str(path[-3:]) + str('_Coronal_') + str(i) + str('.png')
            ruta_2 = str(path) + str('/YOLO/Coronal/Labels/')
            name_2 = str(path[-3:]) + str('_Coronal_') + str(i) + str('.txt')

            #Enventanamos min= -150, max= 250
            img_w = np.clip(img_vol_data[:,i,:], -150, 250)
            #Guardamos normalizada en uint8 en formato .png
            img_norm = (((img_w + 150) / (250 + 150))*255).astype(np.uint8) #Normalizar con escala
            img_n = Image.fromarray(img_norm).convert('L')
            output_path = ruta + name
            img_n.save(output_path)
            
            txt_clase(ruta_2) # Creamos .txt de clase

            with open(str(ruta_2 + name_2), 'w+') as f_txt: # Creamos el resto de .txt
                    for item in (bv_corte[i]): 
                        f_txt.write(str(item) + ' ')
            

        elif corte == 2:
            # Definimos los paths y los nombres para guardar
            ruta = str(path) + str('/YOLO/Axial/Images/')
            name = str(path[-3:]) + str('_Axial_') + str(i) + str('.png')
            ruta_2 = str(path) + str('/YOLO/Axial/Labels/')
            name_2 = str(path[-3:]) + str('_Axial_') + str(i) + str('.txt')

            #Enventanamos min= -150, max= 250
            img_w = np.clip(img_vol_data[:,:,i], -150, 250)
            #Guardamos normalizada en uint8 en formato .png 
            img_norm = (((img_w + 150) / (250 + 150))*255).astype(np.uint8) #Normalizar con escala
            img_n = Image.fromarray(img_norm).convert('L')
            output_path = ruta + name
            img_n.save(output_path)

            txt_clase(ruta_2) # Creamos .txt de clases            
            
            with open(str(ruta_2 + name_2), 'w+') as f_txt: # Creamos el resto de .txt
                for item in (bv_corte[i]): 
                    f_txt.write(str(item) + ' ') 


In [8]:
def txt_clase(path):
    """
    Crea un archivo .txt llamdo clases y escribe en el Nodulo
    - Parámetros:
        path (str): Ruta donde crear el archivo clase.txt
    """
    # Creo, abro y escribo un archivo .txt e introduzco el nombre de la clase
    file_clse = open(path + 'clases.txt',"w") 
    file_clse.write("Nodulo") 
    file_clse.close() 

In [7]:
def path_save(corte):
    """
    Crea el path para guardar las imágenes en un dataset conjunto (imagenes y etiquetas). 
    - Parámetros:
        corte (str): Nombre del corte a guardar (Sagital, Coronal, Axial) 
    - Return:
        path_corte_t_i (str): Path para guardar las imágenes de entrenamiento. 
        path_corte_t_l (str): Path para guardar las etiquetas de entrenamiento.
        path_corte_v_i (str): Path para guardar las imágenes de validación.
        path_corte_v_l (str): Path para guardar las etiquetas de validación.
        path_corte_te_i (str): Path para guardar las imágenes de test.
        path_corte_te_l (str): Path para guardar las etiquetas de test.
    """
    # '/home/dgonzalez2019/NAS/PROYECTOS_INVESTIGACION/2023-RyC-Colon/'
    path_corte_t_i = 'C:/Users/DiegoGO/Desktop/Diego/ING. BIOMÉDICA/5º Carrera/TFG/ProyectoTFG/Yolo_Train/' + corte + '/datasets/newmodel/train/images/'
    path_corte_t_l = 'C:/Users/DiegoGO/Desktop/Diego/ING. BIOMÉDICA/5º Carrera/TFG/ProyectoTFG/Yolo_Train/' + corte + '/datasets/newmodel/train/labels/'
    path_corte_v_i = 'C:/Users/DiegoGO/Desktop/Diego/ING. BIOMÉDICA/5º Carrera/TFG/ProyectoTFG/Yolo_Train/' + corte + '/datasets/newmodel/val/images/'
    path_corte_v_l = 'C:/Users/DiegoGO/Desktop/Diego/ING. BIOMÉDICA/5º Carrera/TFG/ProyectoTFG/Yolo_Train/' + corte + '/datasets/newmodel/val/labels/'
    path_corte_te_i = 'C:/Users/DiegoGO/Desktop/Diego/ING. BIOMÉDICA/5º Carrera/TFG/ProyectoTFG/Yolo_Train/' + corte + '/datasets/newmodel/test/images/'
    path_corte_te_l = 'C:/Users/DiegoGO/Desktop/Diego/ING. BIOMÉDICA/5º Carrera/TFG/ProyectoTFG/Yolo_Train/' + corte + '/datasets/newmodel/test/labels/'
    
    return path_corte_t_i, path_corte_t_l, path_corte_v_i, path_corte_v_l, path_corte_te_i, path_corte_te_l

In [8]:
def path_input(pat, corte_m):
    """
    Crea el path para guardar en el directorio indicado según el corte y el paciente. 
    - Parámetros:
        pat (str): Número del paciente. 
        corte_m (str): Nombre del corte a guardar.
    
    - Return:
        patient_i (str): Path para guardar las imágenes del corte selecionado
        patiente_l (str): Path para guardar las etiquetas del corte selecionado.     
    """
    patient_i = 'C:/Users/DiegoGO/Desktop/Diego/ING. BIOMÉDICA/5º Carrera/TFG/ProyectoTFG/Pacientes_Yolo/' + pat + '/YOLO/' + corte_m + '/Images/'
    patient_l = 'C:/Users/DiegoGO/Desktop/Diego/ING. BIOMÉDICA/5º Carrera/TFG/ProyectoTFG/Pacientes_Yolo/' + pat + '/YOLO/' + corte_m + '/Labels/'

    return patient_i, patient_l

In [11]:
def save_image_labels_train(path_i, path_o):
    """
    Mueve los archivos de un directorio a otro.
    - Parámetros:
        path_i (str): Ruta del directorio de origen.
        path_o (str): Ruta del directorio de destino.
    """
    for file in os.listdir(path_i):
        ruta_file = os.path.join(path_i, file)
        shutil.copy2(ruta_file, path_o)

In [14]:
def img_gran_truth(path_img, path_labels, output_path):
    """
    Procesa imágenes y el GT para dibujar los bounding boxes en las imágenes utilizando cv2. Muestra en pantalla en % de imágenes etiquetadas en la base de datos

    - Parámetros:
        path_img (str): Ruta a las imágenes de entrada.
        path_labels (str): Ruta a los archivos de etiquetas.
        output_path (str): Ruta donde guardar las imágenes con los bounding boxes dibujados.
    """
    img_list = glob.glob(path_img)
    
    img_labwl = 0
    img_total = 0
    for i in img_list:
        image = cv2.imread(i)
        img_height, img_width, _ = image.shape

        name = i.split('/')[-1].split('.')[0]
        fl = open(str(path_labels + name + '.txt'),'r')
        label = fl.read().split(' ')
        fl.close()
        
        if len(label) > 1:
            x = float(label[1])
            y = float(label[2])
            w = float(label[3])
            h = float(label[4])

            l = int((x - w / 2) * img_width)
            r = int((x + w / 2) * img_width)
            t = int((y - h / 2) * img_height)
            b = int((y + h / 2) * img_height)
            
            cv2.rectangle(image, (l, t), (r, b), (0, 255, 0), 2)
            
            img_labwl += 1
        
        file_name = str(name + '.png')
        save_path = os.path.join(output_path, file_name)
        cv2.imwrite(save_path, image)
        img_total += 1
    
    print('Imágenes totales: ', img_total)
    print('Imágenes etiquetadas: ', img_labwl)
    print('Imágenes sin etiquetar: ', img_total - img_labwl,'/','Porcentaje:', round(((img_total - img_labwl) / img_total)*100, 2),'%')

In [2]:
def img_gran_truth_pil(path_img, path_labels, output_path):
    """
    Procesa imágenes y el GT para dibujar los bounding boxes en las imágenes utilizando PIL. Muestra en pantalla en % de imágenes etiquetadas en la base de datos

    - Parámetros:
        path_img (str): Ruta a las imágenes de entrada.
        path_labels (str): Ruta a los archivos de etiquetas.
        output_path (str): Ruta donde guardar las imágenes con los bounding boxes dibujados.
    """

    img_list = glob.glob(path_img)
    
    img_labwl = 0
    img_total = 0
    for i in img_list:
        try:
            image = Image.open(i).convert("RGBA")  # Asegurarse de que la imagen esté en RGBA
            img_width, img_height = image.size

            name = os.path.basename(i).split('.')[0]
            with open(os.path.join(path_labels, name + '.txt'), 'r') as fl:
                label = fl.read().split(' ')
            
            if len(label) > 1:
                x = float(label[1])
                y = float(label[2])
                w = float(label[3])
                h = float(label[4])

                l = int((x - w / 2) * img_width)
                r = int((x + w / 2) * img_width)
                t = int((y - h / 2) * img_height)
                b = int((y + h / 2) * img_height)
                
                draw = ImageDraw.Draw(image)
                draw.rectangle([l, t, r, b], outline="red", width=2)
                
                img_labwl += 1
            
            file_name = f"{name}.png"
            save_path = os.path.join(output_path, file_name)
            image.save(save_path)
            img_total += 1
        
        except Exception as e:
            print(f"Error al procesar la imagen {i}: {e}")
    
    print('Imágenes totales: ', img_total)
    print('Imágenes etiquetadas: ', img_labwl)
    print('Imágenes sin etiquetar: ', img_total - img_labwl, '/', 'Porcentaje:', round(((img_total - img_labwl) / img_total) * 100, 2), '%')

In [ ]:
def gt_voi_in_txt(array, output, patience):
    """
    Guarda el GT del volumen en un archivo de texto como línea de valores separados por espacios para poder almacenarlos.

    - Parámetros:
        array (numpy.ndarray): Array del volumen del paciente.
        output (str): Ruta del directorio de salida.
        patience (str): Identificador del paciente.
    """
    # Convierte el array en una sola línea de texto
    contenido = ' '.join(map(str, array.flatten()))
    
    # Define la ruta de salida usando la variable `pat`
    output_path = os.path.join(output, f"{patience}_gt.txt")
    
    # Guarda el contenido en el archivo de salida
    with open(output_path, 'w') as file:
        file.write(contenido)
    
    print(f"Archivo guardado en: {output_path}")

In [4]:
def px_corners (vol_data, node_coor, axis_index):
    """
    Calcula las coordenadas de los vértices de los bounding boxes en el espacio de píxeles.

    - Parámetros:
        vol_data (numpy.ndarray): Volumen 3D de la imagen médica.
        node_coor (list of numpy.ndarray): Lista de arrays con coordenadas de los nodos detectados.
        axis_index (int): Índice del eje 0: Sagital, 1: Coronal, 2: Axial

    - Return:
        corners_list (list of list): Lista de bounding boxes (4x3).
    """

    corners_list = []
    for i in range(vol_data.shape[axis_index]):
        
         if len(node_coor[i]) != 0:
                x1, x2 = node_coor[i][:, 1].min(), node_coor[i][:, 1].max()
                y1, y2 = node_coor[i][:, 0].min(), node_coor[i][:, 0].max()

                corners = [
                    [int(y1), int(i), int(x1)],
                    [int(y1), int(i), int(x2)],
                    [int(y2), int(i), int(x2)],
                    [int(y2), int(i), int(x1)]
                ] if axis_index == 0 else (
                    [[int(i), int(y1), int(x1)],
                     [int(i), int(y1), int(x2)],
                     [int(i), int(y2), int(x2)],
                     [int(i), int(y2), int(x1)]] if axis_index == 1 else
                    [[int(x1), int(y1), int(i)],
                     [int(x2), int(y1), int(i)],
                     [int(x2), int(y2), int(i)],
                     [int(x1), int(y2), int(i)]]
                )
                
                corners_list.append(corners)
            
    return corners_list

In [3]:
def find_vois(corners_sagital, corners_coronal, corners_axial):
    """
     Encuentra Volúmenes de Interés a partir de las intersecciones de bounding boxes en los tres planos.

    - Parámetros:
        corners_sagital (list of numpy.ndarray): Lista de bounding boxes en el plano sagital.
        corners_coronal (list of numpy.ndarray): Lista de bounding boxes en el plano coronal.
        corners_axial (list of numpy.ndarray): Lista de bounding boxes en el plano axial.

    - Return:
        vois (list of numpy.ndarray): Lista de volúmenes de interés detectados.
    """
    bb_corners_axial = [np.array(bbox) for bbox in corners_axial]
    bb_corners_coronal = [np.array(bbox) for bbox in corners_coronal]
    bb_corners_sagital = [np.array(bbox) for bbox in corners_sagital]
    
    vois = []
    
    for bb_axial in bb_corners_axial:
        in_coronal = False
        in_sagital = False
        for bb_coronal in bb_corners_coronal:
            
            # Verifica que esté dentro de los límites de la caja coronal     
            for_x_axial = bb_axial[:,0].min()<= bb_coronal[:,0].min() <=bb_axial[:,0].max()
            
            for_y_coronal = (bb_coronal[:,1].min()<= bb_axial[:,1].min() <=bb_coronal[:,1].max() or 
                     bb_axial[:,1].min()<= bb_coronal[:,1].max() <=bb_axial[:,1].max())
            
            for_z_coronal = bb_coronal[:,2].min()<= bb_axial[:,2].min() <=bb_coronal[:,2].max()
            
            
            if for_y_coronal and for_z_coronal and for_x_axial: # Dentro del coronal
                in_coronal = True
                
                for bb_sagital in bb_corners_sagital:
                    
                    for_x_sagital = (bb_sagital[:,0].min()<= bb_axial[:,0].min() <=bb_sagital[:,0].max() or                 bb_axial[:,0].min()<= bb_sagital[:,0].max() <=bb_axial[:,0].max())
                    
                    for_z_sagital = bb_sagital[:,2].min()<= bb_axial[:,2].min() <=bb_sagital[:,2].max()
                    
                    for_y_axial = bb_axial[:,1].min()<= bb_sagital[:,1].min() <=bb_axial[:,1].max()
                    
                    if for_x_sagital and for_z_sagital and for_y_axial:
                        in_sagital = True
                        voi = generate_voi(bb_axial, bb_coronal, bb_sagital)
                        if not any(np.array_equal(existing_voi, voi) for existing_voi in vois):
                            vois.append(voi)
    return vois

In [2]:
def full_one_voi(vois):
    """
    Genera un único Volumen de Interés que engloba todos los VOIs individuales.

    - Parámetros:
        vois (list of numpy.ndarray): Lista de VOIs, donde cada VOI es un conjunto de puntos en 3D.
    - Return:
        voi_enclosing_all (numpy.ndarray): Un array (8, 3) con las coordenadas de los vértices del VOI que engloba todos los VOIs.
    """
    all_points = np.vstack(vois)  # Combina todos los puntos en un solo array para facilitar el cálculo

    # Calcula los límites mínimo y máximo en cada dimensión (x, y, z)
    min_x, min_y, min_z = all_points.min(axis=0)
    max_x, max_y, max_z = all_points.max(axis=0)

    # Generar el VOI que engloba a todos los VOIs en `vois`
    voi_enclosing_all = np.array([
        [min_x, min_y, min_z],
        [min_x, min_y, max_z],
        [min_x, max_y, min_z],
        [min_x, max_y, max_z],
        [max_x, min_y, min_z],
        [max_x, min_y, max_z],
        [max_x, max_y, min_z],
        [max_x, max_y, max_z]
    ])

    return voi_enclosing_all

In [3]:
def plot_3d_one_list(coordenadas, color='red'):
    """
    Genera y muestra una visualización 3D de superficies planas a partir de una lista de coordenadas.

    - Parámetros:
        coordenadas (list of list of tuples): Lista de superficies, donde cada superficie es una lista de puntos en 3D.
        color (str, opcional): Color de las superficies en la visualización.
    """

    # Calcular los rangos para cada eje (x, y, z) en función de las coordenadas de entrada
    rangos_ = {
        'x': (min(punto[0] for vertices in coordenadas for punto in vertices),
              max(punto[0] for vertices in coordenadas for punto in vertices)),
        'y': (min(punto[1] for vertices in coordenadas for punto in vertices),
              max(punto[1] for vertices in coordenadas for punto in vertices)),
        'z': (min(punto[2] for vertices in coordenadas for punto in vertices),
              max(punto[2] for vertices in coordenadas for punto in vertices))
    }
    
    # Crear la figura de Plotly
    fig = go.Figure()
    
    # Añadir cada superficie plana a la figura con el color especificado
    for vertices in coordenadas:
        x = [p[0] for p in vertices]
        y = [p[1] for p in vertices]
        z = [p[2] for p in vertices]
        fig.add_trace(go.Mesh3d(
                x=x, y=y, z=z,
                color=color,
                opacity=0.5,
                i=[0, 0, 0, 1, 1, 1],
                j=[1, 2, 3, 2, 3, 0],
                k=[2, 3, 0, 3, 0, 1],
                showlegend=False
            ))
    
    # Ajustar el diseño de la gráfica
    fig.update_layout(scene=dict(
        xaxis=dict(title='X', backgroundcolor='rgba(0,0,0,0)', showgrid=True, range=rangos_['x']),
        yaxis=dict(title='Y', backgroundcolor='rgba(0,0,0,0)', showgrid=True, range=rangos_['y']),
        zaxis=dict(title='Z', backgroundcolor='rgba(0,0,0,0)', showgrid=True, range=rangos_['z'])
    ), 
    title='Superficies planas en el espacio 3D', 
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)')
    
    # Mostrar la figura
    fig.show()

In [3]:
def plot_voi(vois):
    """
    Genera una visualización 3D de un volumen representado como un cubo.

    - Parámetros:
        vois (numpy.ndarray): Array de tamaño (8, 3) que contiene las coordenadas de los 8 vértices del VOI.
    """

    faces = [
    [0, 1, 3, 2],  # Cara del frente
    [4, 5, 7, 6],  # Cara de atrás
    [0, 1, 5, 4],  # Cara de abajo
    [2, 3, 7, 6],  # Cara de arriba
    [0, 2, 6, 4],  # Cara izquierda
    [1, 3, 7, 5]   # Cara derecha
    ]
    
    # Crear la lista de trazas para todos los cubos
    traces = []
    
    # Iterar sobre cada cubo en coordinates
    for cube in [vois]:
        for face in faces:
            # Conectamos los vértices de la cara del cubo actual
            x_vals = [cube[face[0]][0], cube[face[1]][0], cube[face[2]][0], cube[face[3]][0], cube[face[0]][0]]
            y_vals = [cube[face[0]][1], cube[face[1]][1], cube[face[2]][1], cube[face[3]][1], cube[face[0]][1]]
            z_vals = [cube[face[0]][2], cube[face[1]][2], cube[face[2]][2], cube[face[3]][2], cube[face[0]][2]]
            
            traces.append(go.Scatter3d(
                x=x_vals, y=y_vals, z=z_vals,
                mode='lines',
                line=dict(color='rgb(255, 102, 0)', width=4)
            ))
    
    # Definir la figura y añadir todas las trazas de todos los cubos
    fig = go.Figure(data=traces)
    
    # Configuración de la visualización
    fig.update_layout(
        scene=dict(
            xaxis=dict(nticks=4),
            yaxis=dict(nticks=4),
            zaxis=dict(nticks=4)
        ),
        title="VOIs del paciente 068", showlegend=False  # Quitar la leyenda
    )
    
    # Mostrar el gráfico
    fig.show()

In [ ]:
def plot_voi_3_slices(paciente, coordenadas_axial, coordenadas_coronal, coordenadas_sagital, vois):
    """
    Genera una visualización 3D con las superficies axiales, coronales y sagitales, junto con los VOIs.

    - Parámetros:
        paciente (str): Identificador del paciente.
        coordenadas_axial (list of list of tuples): Lista de superficies axiales, donde cada superficie es una lista de puntos en 3D.
        coordenadas_coronal (list of list of tuples): Lista de superficies coronales.
        coordenadas_sagital (list of list of tuples): Lista de superficies sagitales.
        vois (list of numpy.ndarray): Lista de VOIs, donde cada VOI es un conjunto de vértices en 3D.
    """

    faces = [
        [0, 1, 3, 2],  # Cara del frente
        [4, 5, 7, 6],  # Cara de atrás
        [0, 1, 5, 4],  # Cara de abajo
        [2, 3, 7, 6],  # Cara de arriba
        [0, 2, 6, 4],  # Cara izquierda
        [1, 3, 7, 5]   # Cara derecha
    ]

    # Crear la lista de trazas para todos los cubos
    traces = []
    
    # Iterar sobre cada cubo en vois
    for cube in vois:
        for face in faces:
            # Conectamos los vértices de la cara del cubo actual
            x_vals = [cube[face[0]][0], cube[face[1]][0], cube[face[2]][0], cube[face[3]][0], cube[face[0]][0]]
            y_vals = [cube[face[0]][1], cube[face[1]][1], cube[face[2]][1], cube[face[3]][1], cube[face[0]][1]]
            z_vals = [cube[face[0]][2], cube[face[1]][2], cube[face[2]][2], cube[face[3]][2], cube[face[0]][2]]
            
            traces.append(go.Scatter3d(
                x=x_vals, y=y_vals, z=z_vals,
                mode='lines',
                line=dict(color='rgb(255, 102, 0)', width=4)
            ))

    # Crear la figura de Plotly
    fig = go.Figure()

    # Añadir superficies axiales en color rojo
    for vertices in coordenadas_axial:
        x = [v[0] for v in vertices]
        y = [v[1] for v in vertices]
        z = [v[2] for v in vertices]
        
        fig.add_trace(go.Mesh3d(
            x=x, y=y, z=z,
            color='red',
            opacity=0.5,
            i=[0, 1, 2],
            j=[1, 2, 3],
            k=[2, 3, 0],
            showlegend=False
        ))

    # Añadir superficies coronales en color verde
    for vertices in coordenadas_coronal:
        x = [v[0] for v in vertices]
        y = [v[1] for v in vertices]
        z = [v[2] for v in vertices]
        
        fig.add_trace(go.Mesh3d(
            x=x, y=y, z=z,
            color='green',
            opacity=0.5,
            i=[0, 1, 2],
            j=[1, 2, 3],
            k=[2, 3, 0],
            showlegend=False
        ))

    # Añadir superficies sagitales en azul
    for vertices in coordenadas_sagital:
        x = [v[0] for v in vertices]
        y = [v[1] for v in vertices]
        z = [v[2] for v in vertices]
        
        fig.add_trace(go.Mesh3d(
            x=x, y=y, z=z,
            color='blue',
            opacity=0.5,
            i=[0, 1, 2],
            j=[1, 2, 3],
            k=[2, 3, 0],
            showlegend=False
        ))

    # Añadir las trazas de los cubos (VOIs)
    fig.add_traces(traces)
    
    # Ajustar el diseño de la gráfica
    fig.update_layout(scene=dict(
        xaxis=dict(title='X', backgroundcolor='rgba(0,0,0,0)', showgrid=True),
        yaxis=dict(title='Y', backgroundcolor='rgba(0,0,0,0)', showgrid=True),
        zaxis=dict(title='Z', backgroundcolor='rgba(0,0,0,0)', showgrid=True)
    ), 
    title=f'Superficies planas y su VOI en el espacio 3D - Paciente {paciente}', 
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)', showlegend=False)

    # Mostrar la figura en el navegador
    fig.show()

In [ ]:
def volumen_info(x_min, x_max, y_min, y_max, z_min, z_max):
    """
    Calcula el volumen de un cubo en 3D.

    - Parámetros:
        x_min (float): Coordenada mínima en el eje X.
        x_max (float): Coordenada máxima en el eje X.
        y_min (float): Coordenada mínima en el eje Y.
        y_max (float): Coordenada máxima en el eje Y.
        z_min (float): Coordenada mínima en el eje Z.
        z_max (float): Coordenada máxima en el eje Z.

    - Return:
        volumen (int): Volumen del cubo calculado en unidades cúbicas.

    """

    # Ancho, alto y largo del cubo de predicción
    ancho = x_max - x_min
    largo = y_max - y_min
    alto = z_max - z_min
    volumen = int(alto) * int(ancho) * int(largo)

    return volumen